In [1]:
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [2]:
# DATA LOADING

import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date, tzinfo, time
from collections import OrderedDict
from IPython.display import display, clear_output
from IPython.html import widgets
from IPython.display import clear_output
from IPython.display import display, HTML 

# ATENCION -> OTC si no tienen vencimiento listado hay que quitarlas

def date_parser(sDate):
    fmt="%Y-%m-%d %H:%M:%S.%f"
    try: 
        newDate=datetime.strptime(sDate, fmt)
        return newDate
    except:
        return datetime.strptime("1-jan-00", "%d-%b-%y")

# Busca el sabado anterior (es cuando se graba el fichero)
today=date.today()
sat = today - timedelta(today.weekday()+2) # Previous saturday

# FileName & fileURL
fileName="QTPDMkPricesVsTraderPrice" + sat.strftime("%Y%m%d") + ".csv"
fileURL= "H:\MVP00G095\Informes\%s"%fileName

iniData=pd.io.parsers.read_csv(fileURL,thousands=",",decimal=".",sep=";",
                                index_col="ID",parse_dates=[12],
                                date_parser=date_parser,
                                encoding="windows-1250")

# Rename some columns names
iniData=iniData.rename(columns = {'SALES NAME':'Sales N.', 'TRADER NAME':'Trader N.', 
                           'SALES_DESK':'Sales Desk', 'INSTRUMENT GROUP':'Sector',
                           'UNDERLYING_BLOOMBERG':'Underlying', 
                           'CLIENT_FN':'Client', 
                           'TRADER_PREMIUM_BID':'T. Bid', 
                           'TRADER_PREMIUM_OFFER':'T. Offer',
                           'MARKET_BID':'M. Bid',
                           'MARKET_OFFER':'M. Offer',
                           'REQUEST_SENT':'Sent TS'
                           })

# PONER AQUI LAS FECHAS DE ANALISIS
fechaInicio= datetime.strptime("1-jan-17", "%d-%b-%y")
fechaFin= datetime.strptime("31-jan-17", "%d-%b-%y")

# Get data between two dates
mask=(iniData['Sent TS']>=fechaInicio)  & (iniData['Sent TS']<=fechaFin)
data = iniData.loc[mask]

#print "Datos disponibles: %s"%len(fData)

# Remove rows with no market data
data = data[np.isfinite(data['M. Bid'])]
data = data[np.isfinite(data['M. Offer'])]

#print "Datos con market data: %s"%len(fData)

C:\ProgramData\Anaconda2\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
# DATA MANAGEMENT

# Coger el spread de pantalla multiplicado por 1,5 y ver si nuestro
# spread es menor que ese valor
data['marketSpr']=(data['M. Offer'].values-data['M. Bid'].values)*1.5
#data['marketSpr']=(data['M. Offer'].values-data['M. Bid'].-values)
data['traderSpr']=(data['T. Offer'].values-data['T. Bid'].values)
data['spreadIsOK']=(data['traderSpr']<=data['marketSpr'])

# Ver si al menos uno de los precios del trader entra en la horquilla
# Para operaciones que no sean OOF
data['priceIsInRange']=((data['T. Offer'].values<=data['M. Offer'].values) & \
    (data['T. Offer'].values>data['M. Bid'].values)) | \
    ((data['T. Bid'].values>=data['M. Bid'].values) & \
    (data['T. Bid'].values<data['M. Offer'].values))
    
# Ver si cumple las dos condiciones
data['priceIsOK']=(data['spreadIsOK']&data['priceIsInRange'])

def muestraInforme():
    clear_output() # Borra informe anterior
    
    # Muestra informe
    
    # Título
    display(HTML("<h1>INFORME DE LA CALIDAD DE LOS PRECIOS</h1>\n")) 

    # ANALISIS DE LOS DATOS SELECCIONADOS
    # Porcentaje de operaciones que cumplen el framework
    porcentaje=data['priceIsOK'].mean()
    print ""
    print "Nº operaciones = %s"%len(data)
    print "Porcentaje de operaciones correctas = %.2f"%(porcentaje*100)  
    
    displayData=data[['Sent TS', 'Underlying','T. Bid', 'T. Offer', 'M. Bid', 'M. Offer', 'priceIsOK']]
    display(HTML(displayData.to_html()))

muestraInforme()


Nº operaciones = 935
Porcentaje de operaciones correctas = 72.09


,Sent TS,Underlying,T. Bid,T. Offer,M. Bid,M. Offer,priceIsOK
ID,,,,,,,
79216,2017-01-02 09:06:10,SX5E INDEX,24.500000,26.000000,24.64535,25.92035,False
79218,2017-01-02 09:41:44,SX5E INDEX,60.200000,61.600000,60.02826,61.52826,True
79220,2017-01-02 10:57:23,ENI IM EQUITY,0.714000,0.753000,0.72000,0.74150,False
79221,2017-01-02 11:02:06,ENI IM EQUITY,0.300162,0.334965,0.30361,0.32211,False
79222,2017-01-02 11:09:10,SX5E INDEX,11.800000,12.700000,11.82162,12.42162,False
79224,2017-01-02 12:43:55,ISP IM EQUITY,0.210550,0.213985,0.21015,0.21690,True
79226,2017-01-02 13:47:24,SAP GY EQUITY,23.510000,23.550000,22.69284,23.67284,True
79228,2017-01-02 14:11:42,SX7E INDEX,3.660000,3.800000,3.61471,3.86471,True
79230,2017-01-02 14:34:48,FCA IM EQUITY,0.572389,0.580083,0.57432,0.59382,True
